In [ ]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

In [2]:
from models.horse import Horse
from database.setup import create_sqlalchemy_session
import tqdm
import re

from utils.music import parse_music

BATCH_SIZE = 1e5
with create_sqlalchemy_session() as db_session:
    horse=db_session.query(Horse).first()

In [3]:
start_id=0
with create_sqlalchemy_session() as db_session:
    wrong_horse_ids=set()

    horses=db_session.query(Horse).filter(Horse.id>start_id).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
    for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=BATCH_SIZE):
        ages = [runner.age for runner in horse.runners if runner.age is not None]
        if sorted(ages) != ages:
            wrong_horse_ids.add(horse.id)
            
    while len(horses)==BATCH_SIZE:
        start_id=horses[-1].id
        horses=db_session.query(Horse).filter(Horse.id>start_id).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
        for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=len(horses)):
            ages = [runner.age for runner in horse.runners if runner.age is not None]
            if sorted(ages) != ages:
                wrong_horse_ids.add(horse.id)

0:   0%|          | 0/100000.0 [00:00<?, ?it/s]


In [4]:
len(wrong_horse_ids)

0

In [5]:
162684 in wrong_horse_ids

False

In [6]:
start_id=0
with create_sqlalchemy_session() as db_session:

    horses=db_session.query(Horse).filter(Horse.id>start_id).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
    
    for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=BATCH_SIZE):
        origins={runner.origins for runner in horse.runners if runner.origins is not None}
        if not origins:
            continue
        if len(origins)==2:
            continue
        print(origins, horse.id)
        break

0:   0%|          | 0/100000.0 [00:00<?, ?it/s]


In [7]:
a="Turgeon-Shannon"

In [8]:
father_name, mother_name=re.split("[-/]",a)

In [9]:
father_name=father_name.upper()
mother_name=mother_name.upper()

In [10]:
with create_sqlalchemy_session() as db_session:

    father_horse_candidates=db_session.query(Horse).filter(Horse.name==father_name).all()
    
    mother_horse_candidates=db_session.query(Horse).filter(Horse.name==mother_name).all()

In [11]:
mother_horse_candidates

[]

In [12]:
from collections import namedtuple
from typing import List

MusicEvent = namedtuple("MusicEvent", ["rank", "race_type", "year"])


def parse_unibet_music(current_year:int, ) ->List[MusicEvent]:

    music_events:List[MusicEvent]=[]
    events=re.findall(r"(([ATRDN]|\d)([a-z]))|(\((\d{1,2})\))", runner.music)
    for event in events:
        race_event, rank, race_type, _, year = event
        if year:
            assert not race_event
            next_current_year = int(f'20{year}')
            # We read music from left to right to go backward in time
            assert next_current_year < current_year
            current_year=next_current_year
            continue
        assert not year
        music_events.append(MusicEvent(rank=rank, race_type=race_type, year=current_year))
        
    return music_events
    

In [13]:
music_events

NameError: name 'music_events' is not defined

In [ ]:
runner.music

In [ ]:
race_type

In [ ]:
_

In [ ]:
year

In [ ]:
race_type

In [ ]:
import logging
from collections import Counter
import datetime as dt
from typing import Optional, Generator
from constants import UNIBET_MIN_DATE
from tqdm import tqdm
import os 
import json

logger = logging.getLogger(__name__)
UNIBET_DATA_PATH = "./data/Unibet"

sexes=Counter()

def date_countdown_generator(
    start_date: dt.date, end_date: Optional[dt.date]
) -> Generator[dt.date, None, None]:
    end_date = end_date or dt.date.today()
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += dt.timedelta(days=1)
        

for date in tqdm(
    date_countdown_generator(
        start_date=UNIBET_MIN_DATE,
        end_date=dt.date.today() - dt.timedelta(days=1),
    ),
    total=(dt.date.today() - dt.timedelta(days=1) - UNIBET_MIN_DATE).days,
    unit="days",
):
    if not date.isoformat() in os.listdir(UNIBET_DATA_PATH):
        logger.info("Could not find folder for date: %s", date.isoformat())
        continue
    day_folder_path = os.path.join(UNIBET_DATA_PATH, date.isoformat())
    if "programme.json" not in os.listdir(day_folder_path):
        logger.info(
            "Could not find programme.json for date: %s", date.isoformat()
        )
        continue

    with open(os.path.join(day_folder_path, "programme.json"), "r") as fp:
        programme = json.load(fp=fp)
    if "data" not in programme:
        logger.info("Can not import programme of %s", date.isoformat())
        continue

    horse_shows_dict = programme["data"]
    for horse_show_dict in horse_shows_dict:

        if horse_show_dict.get("races"):

            for race_dict in horse_show_dict["races"]:
                race_path = os.path.join(
                    day_folder_path,
                    f"R{horse_show_dict['rank']}_C" f"{race_dict['rank']}.json",
                )
                with open(race_path, "r") as fp:
                    complete_race_dict = json.load(fp=fp)

                current_race_dict = complete_race_dict
                if complete_race_dict.get("note") == "server error, no json":
                    # Can not use complete_race
                    current_race_dict = race_dict

                for runner_dict in current_race_dict["runners"]:
                    sexes[runner_dict["details"].get("sex") if runner_dict.get("details") else None]+=1

In [ ]:
sexes

In [ ]:
from models.horse import Horse